In [ ]:
!pip install tensorflow numpy pandas dd pyeda

In [ ]:
# Required Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Read the CSV data
data = pd.read_csv('inputs_2exp17.csv') # 131,072

# Data processing
expressions = data['expression'].tolist()
reorderings = data['reordering'].tolist()

In [ ]:
print(data[10:15])

   expression ordering  bdd_size reordering  bdd_size_reordered
10    c|d|a&f  d:f:a:c        10    d:f:a:c                   5
11    c|b&f&e  f:e:c:b        11    e:f:b:c                   5
12    a|c|e&f  a:e:f:c        10    a:e:f:c                   5
13    f|b|c|e  b:c:f:e        11    c:b:f:e                   5
14    b|a&d|d    a:d:b         7      a:d:b                   3


In [ ]:
# Prepare the tokenizers
# For input expressions (characters)
input_tokenizer = Tokenizer(char_level=True, filters='')
input_tokenizer.fit_on_texts(expressions)
input_vocab_size = len(input_tokenizer.word_index) + 1  # +1 for padding token
print(input_tokenizer.word_index)

# For output reorderings (tokens separated by ':')
# We will replace ':' with space to tokenize the variables
target_texts = []
for seq in reorderings:
    # Add start and end tokens for the decoder
    target_texts.append('<start> ' + seq.replace(':', ' ') + ' <end>')

# Prepare tokenizer for outputs
output_tokenizer = Tokenizer(filters='')
output_tokenizer.fit_on_texts(target_texts)
output_vocab_size = len(output_tokenizer.word_index) + 1  # +1 for padding token
print(input_vocab_size, output_vocab_size)
print(target_texts)
print(output_tokenizer.word_index)

In [ ]:
# Convert expressions and reorderings to sequences
encoder_input_sequences = input_tokenizer.texts_to_sequences(expressions)

decoder_input_texts = []
decoder_target_texts = []
for seq in reorderings:
    decoder_input_texts.append('<start> ' + seq.replace(':', ' '))
    decoder_target_texts.append(seq.replace(':', ' ') + ' <end>')

decoder_input_sequences = output_tokenizer.texts_to_sequences(decoder_input_texts)
decoder_target_sequences = output_tokenizer.texts_to_sequences(decoder_target_texts)
print(decoder_input_sequences[0:3])
print(decoder_target_sequences[0:3])

# Determine the maximum sequence lengths
max_encoder_seq_length = max([len(seq) for seq in encoder_input_sequences])
max_decoder_seq_length = max([len(seq) for seq in decoder_input_sequences])
print(max_encoder_seq_length, max_decoder_seq_length)

# Pad the sequences
encoder_input_data = pad_sequences(encoder_input_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_sequences, maxlen=max_decoder_seq_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_sequences, maxlen=max_decoder_seq_length, padding='post')

[[1, 7, 6, 4], [1, 3, 8, 7, 6], [1, 8, 3, 7, 6]]
[[7, 6, 4, 2], [3, 8, 7, 6, 2], [8, 3, 7, 6, 2]]
9 5


In [ ]:
# Define the embedding dimension and latent dimension
embedding_dim = 50
latent_dim = 256

# Build the encoder
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
enc_emb = Embedding(input_dim=input_vocab_size, output_dim=embedding_dim, name='encoder_embedding')(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Build the decoder
decoder_inputs = Input(shape=(None,), name='decoder_inputs')
dec_emb_layer = Embedding(input_dim=output_vocab_size, output_dim=embedding_dim, name='decoder_embedding')
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# Add an additional dimension to decoder_target_data for sparse_categorical_crossentropy
decoder_target_data = np.expand_dims(decoder_target_data, -1)

# Train the model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=128,
    epochs=100,
    validation_split=0.2
)

Epoch 1/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 99s 117ms/step - loss: 1.0978 - val_loss: 0.4809
Epoch 2/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 139s 114ms/step - loss: 0.4677 - val_loss: 0.4669
Epoch 3/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 142s 114ms/step - loss: 0.4580 - val_loss: 0.4538
Epoch 4/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 142s 114ms/step - loss: 0.4437 - val_loss: 0.4308
Epoch 5/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 143s 115ms/step - loss: 0.4044 - val_loss: 0.3890
Epoch 6/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 140s 113ms/step - loss: 0.3728 - val_loss: 0.3737
Epoch 7/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 142s 113ms/step - loss: 0.3553 - val_loss: 0.3521
Epoch 8/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 93s 113ms/step - loss: 0.3456 - val_loss: 0.3476
Epoch 9/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 142s 113ms/step - loss: 0.3389 - val_loss: 0.3436
Epoch 10/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 143s 115ms/step - loss: 0.3354 - val_loss: 0.3422
Epoch 11/100
820/820 ━━━━━━━━━━━━━━━━━━━━ 140s 113ms/step - loss: 0.3336 - val_loss: 0.3405

KeyboardInterrupt: 

In [ ]:
# Inference models for testing
# Build the encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Build the decoder model
decoder_state_input_h = Input(shape=(latent_dim,), name='decoder_state_input_h')
decoder_state_input_c = Input(shape=(latent_dim,), name='decoder_state_input_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

# Mapping indices back to words
reverse_input_char_index = {v: k for k, v in input_tokenizer.word_index.items()}
reverse_target_word_index = {v: k for k, v in output_tokenizer.word_index.items()}

# Function to decode the sequence
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start token.
    target_seq = np.array([output_tokenizer.word_index['<start>']])
    target_seq = np.expand_dims(target_seq, 0)

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index.get(sampled_token_index, '')

        if sampled_word == '<end>' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + ' '

            # Update the target sequence
            target_seq = np.array([[sampled_token_index]])

            # Update states
            states_value = [h, c]

    return decoded_sentence.strip()

 6380/17162 ━━━━━━━━━━━━━━━━━━━━ 59s 6ms/step - loss: 1.1618

KeyboardInterrupt: 

In [ ]:
filtered_data = data[data['expression'] == 'a&b|c&d']
print(filtered_data)
# write code to get all the rows where data['expression'] == 'a|b&c'

       expression ordering  bdd_size reordering  bdd_size_reordered
4521      a&b|c&d  c:a:d:b        10    c:d:b:a                   5
4577      a&b|c&d  a:b:c:d         9    a:b:d:c                   5
9532      a&b|c&d  b:c:d:a        10    a:b:c:d                   5
14203     a&b|c&d  c:b:a:d        10    a:b:c:d                   5
18653     a&b|c&d  b:a:c:d         9    a:b:d:c                   5
39999     a&b|c&d  d:b:a:c        10    a:b:d:c                   5
42557     a&b|c&d  a:c:d:b        10    a:b:c:d                   5
43935     a&b|c&d  c:d:a:b         9    c:d:b:a                   5
44120     a&b|c&d  d:c:a:b         9    c:d:b:a                   5
44857     a&b|c&d  b:c:a:d        10    a:b:c:d                   5
56873     a&b|c&d  a:c:b:d        10    a:b:c:d                   5
60023     a&b|c&d  a:b:d:c         9    a:b:d:c                   5
67415     a&b|c&d  d:c:b:a         9    c:d:b:a                   5
78527     a&b|c&d  b:a:d:c         9    a:b:d:c 

In [ ]:
test_expression = ['a&b|c&d', 'a&b']
test_sequence = input_tokenizer.texts_to_sequences([test_expression])
test_sequence = pad_sequences(test_sequence, maxlen=max_encoder_seq_length, padding='post')
decoded_ordering = decode_sequence(test_sequence)
print('Predicted variable ordering:', decoded_ordering)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted variable ordering: b f a
